# Search_pdf_on_GALLICA_from_a_list
* En entrée : liste de titres de documents à rechercher sur Gallica
* En sortie : un fichier avec la liste de tous les titres trouvés et leurs url + un fichier avec la même liste et un comparateur de similarité pour confronter les résultats avec les titres requêtés.

In [24]:
#Import des librairies nécessaires
import os
import re
import urllib
import lxml.etree as etree
import glob
import jellyfish

## Lancer l'API GALLICA et pré-filtrer les résultats
* Format de fichier en entrée : .txt ou .tsv 

* une ligne = identification_du_document (tabulation) titre_à_chercher

In [18]:
def load_clean_tronc(path, erase_ponctuation = True, nb_mots = 10):
#Pour charger les titres à requêter, supprimer la ponctuation et les couper car les titres longs 
#sont plus susceptibles de générer de mauvais résultats dans l'api.
#Path = chemin vers le fichier où sont les titres
#nb_mots = nombre de mots à garder pour les titres-requêtes
#erase_ponctuation = True pour supprimer la ponctuation"""
    with open(path, "r", encoding='utf8') as f :
        lignes=f.readlines()
    if erase_ponctuation == True :
        lignes_clean = [re.sub(" \([\w]+[\'|\’]?\)", "", i) for i in lignes]
        lignes_clean = [re.sub('[^\w\t ]+', ' ', i) for i in lignes_clean]
        lignes_clean = [re.sub('  ', ' ', i) for i in lignes_clean]
        lignes_clean = [re.sub("\n", '', i) for i in lignes_clean]
    else :
        lignes_clean=lignes
    liste_tronquee=[]
    for l in lignes_clean:
        try:
            idMoreau, titre=re.split("\t",l)
        except:
            print("erreur à la ligne suivante : "+l)
        if len(titre)<40:
            liste_tronquee.append(idMoreau+"\t"+titre)
        if len(titre)>40:
            try:
                titre_coupe=re.split(" ",titre)
            except:
                print("erreur dans le titre suivant : "+titre)
            titre_tronque=titre_coupe[0:nb_mots]
            titre_tronque_str = '%20'.join(titre_tronque)
            liste_tronquee.append(idMoreau+"\t"+titre_tronque_str)
    return liste_tronquee

In [19]:
def launch_api_gallica(path, max_results = 40):
#Cette fonction transforme les titres en requête URL valide dans l'api gallica, puis enregistre les résultats de
#l'api dans un fichier XML (identifiant_du_titre.xml) 
#paramètres API gallica : cherche toutes les monographies contenant les mots du titre, publiées au XVIIe siècle
    liste_tronquee = load_clean_tronc(path, erase_ponctuation = True, nb_mots = 10)
    try:
        os.mkdir('sortie_xml')
    except:
        pass
    chaine1 = "https://gallica.bnf.fr/SRU?operation=searchRetrieve&version=1.2&query=%20gallica%20any%20%22"
    chaine2 = "%22%20and%20dc.type%20any%20monographie%20and%20(century%20adj%20%2217%22)&maximumRecords="
    chaine3 = "%s" %max_results
    list_no_results=[]
    for l in liste_tronquee:
        idMoreau, titre=re.split("\t",l)
        titre_ascii=urllib.parse.quote(titre) #transformer les titres au format ascii
        url= chaine1 + titre_ascii + chaine2 + chaine3
        try:
            txt = urllib.request.urlretrieve(url, 'sortie_xml/%s.xml' %idMoreau)
        except:
            list_no_results.append(l)
    if not(list_no_results):
        print("Tous les titres ont été requêtés et les résultats enregistrés dans des XML.")
    else:
        #relancer l'api sur les titres qui n'ont pas pu être requêtés avec une autre clé GBOOKS:
        print("Nombre de titres pour lesquels la recherche API n'a pas fonctionné : "+str(len(list_no_results))+". Liste des titres enregistrée au format .txt au même niveau que ce script.")

In [20]:
def download_filtered_url(nombre = 40, regex="1648|1649|1650|1651|1652|1653|1654"):
#enregistre dans une liste les résultats de l'api correspondants à la recherche avec : 
#date de publi + titre du document + adresse URL où télécharger le doc
#Enregistre les résultats au format .txt
#nombre = nombre max de résultats de la requête de l'api selectionné
#regex = mettre la date ou les dates de publication qui serviront à filtrer les résultats
    p = re.compile("%s" %regex,re.IGNORECASE)
    liste_url=[]
    for filepath in glob.iglob('sortie_xml/*.xml'):
            if filepath.endswith('.xml'):
                tree = etree.parse("%s" %filepath)
                for i in range(nombre): 
                    for date in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[%s]/srw:recordData[1]/oai_dc:dc[1]/dc:date[1]" %str(i+1),
                              namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                        m = p.search(date.text)
                        if m:
                            v_date=m.group()
                            for title in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[%s]/srw:recordData[1]/oai_dc:dc[1]/dc:title[1]" %str(i+1),
                                       namespaces={'srw': 'http://www.loc.gov/zing/srw/','oai_dc':"http://www.openarchives.org/OAI/2.0/oai_dc/",'dc':'http://purl.org/dc/elements/1.1/'}):
                                v_titre=title.text
                            for link in tree.xpath("/srw:searchRetrieveResponse/srw:records[1]/srw:record[%s]/srw:extraRecordData[1]/link[1]" %str(i+1),
                                      namespaces={'srw': 'http://www.loc.gov/zing/srw/'}):
                                v_url=link.text
                            ligne=str(v_date)+"\t"+v_titre+"\t"+v_url
                            liste_url.append(ligne)
    liste_url=list(set(liste_url))
    print("Nombre de fichiers correspondants trouvés : " + str(len(liste_url)))
    try:
        os.mkdir('output')
    except:
        pass
    open("output/liste_url.txt", "w", encoding="utf8")
    for l in liste_url :
        with open ("output/liste_url.txt", "a", encoding="utf8") as f:
            f.write(l+"\n"+"\n")
    return liste_url

## Passer le résultat de l'API dans le get_similar_titles

In [21]:
def get_tsv(path):
    f = open(path,encoding="utf-8")
    liste = f.readlines()
    f.close()
    return liste

def get_closest(path,title, number = 5):
#En entrée, une chaîne de caractère (et en option le nombre de titres proches à retourner, par défaut : 5)
#En sortie, la liste triée par ordre décroissant de similarité sous forme d'un triplet:
#distance, ID_moreau, titre concerné.
    liste = get_tsv(path)
    mots_titre = set(title.split())
    filtered_list = []
    for l in  liste:
        ID, chaine=re.split("\t",l)
        mots_candidat = set(chaine.split())
        inter = mots_titre.intersection(mots_candidat)
        ##Si moins de 5 mots en commun (ou de la moitié) inutile de garder le candidat
        if len(inter)>5 or len(inter)>(len(mots_titre)/2):
            filtered_list.append([len(inter), ID, chaine])
    f2 = []
    for len_inter, ID, chaine in filtered_list:
        sim = jellyfish.jaro_winkler(title,chaine)
        ## On garde quand la similarité est supérieure à 0.5
        if sim>0.5:
            f2.append([sim, ID, chaine])
    return sorted(f2, reverse=True)[:number]

In [22]:
def download_results(path,nombre = 40, regex="1648|1649|1650|1651|1652|1653|1654"):
#nombre = nombre de sortie api gallica à intégrer dans les résultats enregistrés pour une requête
#enregistrer les résultats dans un fichier .txt à ouvrir avec notepad++ pour plus de lisibilité
#regex=mettre une ou des dates de publication (séparées par des "|") pour filtrer les résultats
    liste_titres = download_filtered_url(nombre, regex)
    dic= {}
    cpt = 0
    open ("output/output_similar.txt", "w", encoding="utf8")
    for lt in liste_titres: #A titre d'exemple
        date, titre,url=re.split("\t",lt)
        with open ("output/output_similar.txt", "a", encoding="utf8") as f:
            f.write("\n"+"-"*30+"\n"+ date + "\t"+ titre + "\n"+ url+"\n")
        print("-"*30)
        print(date, titre, url)
        closest_titles = get_closest(path,titre)
        for dist, ID_moreau, chaine in closest_titles:
            result=f"  D={round(dist, 4)} (ID={ID_moreau}): {chaine[:70]}"
            print(result)
            with open ("output/output_similar.txt", "a", encoding="utf8") as f:
                f.write (result+"\n")

# Exemple d'utilisation :
* Remplacer "essai_titre.txt" par le nom de votre fichier avec les titres à chercher en respectant le format (id_titre+tabulation+titre+saut de ligne). Veiller à ce qu'il se trouve bien au même niveau que ce script.
* Cliquer sur Cell > Run all. Attendre quelques minutes.
* L'ensemble des résultats de l'application se trouvent dans le dossier de sortie "output", créé au même niveau que le script.
* Conseil : ouvrir le fichier "output_similar.txt" avec le logiciel notepad++ pour une meilleure lisibilité.

In [15]:
launch_api_gallica("CoulonTitres.txt",max_results = 40)

Nombre de titres pour lesquels la recherche API n'a pas fonctionné : 1


TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

In [25]:
download_results("CoulonTitres.txt", nombre = 40, regex="1648|1649|1650|1651|1652|1653|1654")

Nombre de fichiers correspondants trouvés : 315
------------------------------
1648 Le Virgile travesty en vers burlesques , de M. Scarron. Dédié à la Reyne https://gallica.bnf.fr/ark:/12148/bpt6k123053b
  D=0.567 (ID=106): Recit du duel memorable fait à Ruel entre dix laquais des deputez, & a
------------------------------
1651 Réflexions sur l'arest du Parlement donné en faveur de Mr le Duc de Vendosme contre les prétensions de Mr le Duc d'Elboeuf https://gallica.bnf.fr/ark:/12148/bpt6k5425495x
------------------------------
1649 Avis+ heroïques et importants donnez à Mr le prince de Condé # par monsieur de Chastillon, revenu de l'autre monde . Par l'autheur mesme des Triolets [du prince de Condé] https://gallica.bnf.fr/ark:/12148/bpt6k5661053j
------------------------------
1653 Ismaelis Bullialdi observatio secundi deliquii lunaris anno 1652 mense septembri facti, una cum calculo illius, et futuri alius lunae defectus mense martio 1653 ex tabulis Philolaicis, et Observationes circa

  D=0.7265 (ID=160): Les particularitez du second combat donné entre l'armée de son Altesse
  D=0.7139 (ID=136): Lettre de M. d'Auremesnil, chef de la noblesse de Caux en Normandie, e
  D=0.6796 (ID=120): Manifeste pour M. le duc de Bouillon et messieurs les autres généraux 
  D=0.674 (ID=162): Recit veritable de tout ce qui s'est passé à l'Hostel de Ville touchan
  D=0.6577 (ID=106): Recit du duel memorable fait à Ruel entre dix laquais des deputez, & a
------------------------------
1649 Le portraict de l'inconstance des armes https://gallica.bnf.fr/ark:/12148/bpt6k56119412
------------------------------
1649 Le Mazarin portant la hotte, dit, J'ay bon dos, je porteray bien tout https://gallica.bnf.fr/ark:/12148/bpt6k5414619n
------------------------------
1651 Les particularitez de tout ce qui s'est fait et passé à la dispute et conversion à la foy catholique, apostolique & romaine de monsieur le marquis de Meillars , mareschal des camps & armées du roy, de sa femme & de ses enfans, 

  D=0.7413 (ID=117): Lettre déchiffrée contenant plusieurs avis qu'un des émissaires et esp
  D=0.724 (ID=96): Lettre déchiffrée, contenant plusieurs advis qu’un des émissaires et e
  D=0.7158 (ID=98): Lettre contenant ce qui s’est passé en l’Assemblée du Parlement, depui
  D=0.7116 (ID=115): Lettre contenant ce qui s'est passé en l'assemblée du Parlement, depui
------------------------------
1652 Le caractère de la royauté et de la tyrannie, faisant voir par un discours politique https://gallica.bnf.fr/ark:/12148/bpt6k800834
------------------------------
1652 Les presens faits à Son A. royalle et à Messieurs les Princes de Condé et de Beaufort à leur arrivée au Palais. Pour Messieurs du Parlement https://gallica.bnf.fr/ark:/12148/bpt6k57085353
  D=0.6704 (ID=125): Requête (la) des bourgeois de Paris à nosseigneurs du Parlement toucha
------------------------------
1649 Hircanus tragoedia dabitur in Theatrum Regiae Navarrae ad solennem praemiorum distributionem ex munificentia Ludovic

------------------------------
1650 L'héritier ridicule, ou La dame intéressée , comédie dédiée au prince d'Orange, par M. Scarron https://gallica.bnf.fr/ark:/12148/bpt6k717535
------------------------------
1650 (Pierre de Provence.) Aci comensa la historia del noble y esforçat Cavaller Pierres de Provença, fill del comte de Provença, y de la gentil Magalona filla del rey de Napole, y de les fortunes y treballo que passaren en la sua molt enamorada vida, traduyda de llengua castellana, en rostra llengua catalana per la discret y honrat Honorat Comalada https://gallica.bnf.fr/ark:/12148/bpt6k8573996
------------------------------
1654 Arrest du Conseil d'estat, portant décry de tous les deniers fabriquez sous le nom du prince de Mantoüe, & autres deniers etrangers ; dont l'emprainte est cy représentée https://gallica.bnf.fr/ark:/12148/bpt6k1189563n
------------------------------
1652 Histoire de Magdelaine Bavent, religieuse du monastère de Saint-Louis de Louviers, avec sa confession g

  D=0.6389 (ID=162): Recit veritable de tout ce qui s'est passé à l'Hostel de Ville touchan
  D=0.6176 (ID=154): La harangue du roy de la Grand Bretagne a la reyne de France, faicte a
------------------------------
1650 Decret ... donne en l'assemblee generale de nosseigneurs les eminentissimes et reverendissimes cardinaux contre le catechisme des Jansenistes, et excommunication de ses autheurs https://gallica.bnf.fr/ark:/12148/bpt6k5340260s
  D=0.6525 (ID=120): Manifeste pour M. le duc de Bouillon et messieurs les autres généraux 
------------------------------
1651 Reglement general, concernant les attaches des batteaux, vente, debite, mesure & ports des charbons (4 mai 1651) https://gallica.bnf.fr/ark:/12148/bpt6k101164b
------------------------------
1648 Polyeucte martyr, tragédie de Monsr Corneille https://gallica.bnf.fr/ark:/12148/bpt6k12803108
  D=0.6355 (ID=54): Corneille (Pierre), Polyeucte martyr, Paris, Antoine de Sommaville et 
------------------------------
1649 Plaisant 

  D=0.721 (ID=98): Lettre contenant ce qui s’est passé en l’Assemblée du Parlement, depui
  D=0.7172 (ID=115): Lettre contenant ce qui s'est passé en l'assemblée du Parlement, depui
  D=0.711 (ID=134): Extrait d’une lettre envoyée de Ruel, en date du vendredi 19° de ce mo
  D=0.7091 (ID=96): Lettre déchiffrée, contenant plusieurs advis qu’un des émissaires et e
  D=0.7008 (ID=120): Manifeste pour M. le duc de Bouillon et messieurs les autres généraux 
------------------------------
1651 Les confessions de S. Augustin, traduites en françois par Monsieur Arnauld d'Andilly . Quatriesme edition. Avec le latin à costé, reveu & corrigé exactement sur douze anciens manuscrits, & des notes à la fin... Par Monsieur Arnauld son frere docteur en theologie... https://gallica.bnf.fr/ark:/12148/bpt6k15101126
  D=0.7644 (ID=98): Lettre contenant ce qui s’est passé en l’Assemblée du Parlement, depui
  D=0.7617 (ID=136): Lettre de M. d'Auremesnil, chef de la noblesse de Caux en Normandie, e
  D=0.7339 

  D=0.7206 (ID=106): Recit du duel memorable fait à Ruel entre dix laquais des deputez, & a
  D=0.7038 (ID=128): Royauté (la) de Charles second , roi de la Grand’ Bretagne, etc., reco
  D=0.6995 (ID=136): Lettre de M. d'Auremesnil, chef de la noblesse de Caux en Normandie, e
  D=0.6942 (ID=134): Extrait d’une lettre envoyée de Ruel, en date du vendredi 19° de ce mo
------------------------------
1653 Factum pour messieurs Le Maistre, Mangot, de Creil, maîtres des requêtes, messieurs Charlet, Baron, Bitault, Perrot, Foucault, Barentin, conseillers, en la cour, et plus de cent créanciers,... contre Me Nicolas Berthault, receveur des consignations des Requêtes du Palais, Me Charles Bernage et consorts... / (Signé : Cousturier.) https://gallica.bnf.fr/ark:/12148/bpt6k8522458
  D=0.7078 (ID=120): Manifeste pour M. le duc de Bouillon et messieurs les autres généraux 
  D=0.7058 (ID=162): Recit veritable de tout ce qui s'est passé à l'Hostel de Ville touchan
  D=0.684 (ID=154): La harangue du

------------------------------
1649 Les Comme et ainsi de la cour https://gallica.bnf.fr/ark:/12148/bpt6k56622828
  D=0.6465 (ID=163): Les Coups de l’Amour et de la Fortune, Paris, Guillaume de Luyne, 1655
  D=0.577 (ID=158): Les Délibérations prises et arrêtées en l’Hôtel de Ville pour la levée
------------------------------
1650 L'Ovide en belle humeur de Mr Dassoucy . Enrichy de toutes ses figures burlesques https://gallica.bnf.fr/ark:/12148/bpt6k1168131
------------------------------
1650 Histoire du ministère d'Armand-Jean Du Plessis, cardinal, duc de Richelieu, sous le règne de Louys le Juste, XIII. du nom,... avec des réflexions politiques, et diverses lettres, contenant les négociations des affaires du Piedmont et du Montferrat https://gallica.bnf.fr/ark:/12148/bpt6k8717027d
  D=0.7309 (ID=96): Lettre déchiffrée, contenant plusieurs advis qu’un des émissaires et e
  D=0.7185 (ID=117): Lettre déchiffrée contenant plusieurs avis qu'un des émissaires et esp
  D=0.6985 (ID=120): Ma

------------------------------
1651 Plaintes de la France à ses peuples sur l'emprisonnement des Princes contre Mazarin https://gallica.bnf.fr/ark:/12148/bpt6k5725183d
------------------------------
1650 Estrennes burlesques pour le premier jour de l'an 1650 https://gallica.bnf.fr/ark:/12148/bpt6k5700628t
------------------------------
1649 Harangue faite à la reine, à Amiens . Par E. P. https://gallica.bnf.fr/ark:/12148/bpt6k8501742
------------------------------
1649 Saint Eustache martyr , poëme dramatique de Baro https://gallica.bnf.fr/ark:/12148/bpt6k116953q
------------------------------
1649 Triolets sur la conference tenuë à Ruel https://gallica.bnf.fr/ark:/12148/bpt6k5429363v
------------------------------
1651 Cartel aux bons François, pour la majorité du Roy https://gallica.bnf.fr/ark:/12148/bpt6k5426113b
------------------------------
1648 Les devoirs mutuels des grands seigneurs, et de ceux qui les servent, ou L'art de vivre à la cour, & de converser avecque les grands . E

  D=0.7945 (ID=136): Lettre de M. d'Auremesnil, chef de la noblesse de Caux en Normandie, e
  D=0.7921 (ID=160): Les particularitez du second combat donné entre l'armée de son Altesse
  D=0.7622 (ID=154): La harangue du roy de la Grand Bretagne a la reyne de France, faicte a
  D=0.7258 (ID=106): Recit du duel memorable fait à Ruel entre dix laquais des deputez, & a
  D=0.7211 (ID=162): Recit veritable de tout ce qui s'est passé à l'Hostel de Ville touchan
------------------------------
1649 Visions astrologiques de Michel Nostradamus sur toutes les affaires de ce temps, et la confusion de Mazarin . En vers burlesques https://gallica.bnf.fr/ark:/12148/bpt6k118539m
  D=0.6727 (ID=118): Lettre du sieur de Nacar à l'abbé de La Rivière, à Saint-Germain-en-La


**L'ensemble des résultats de l'application se trouvent dans le dossier de sortie "output".**